In [1]:
import sys

from tigramite import data_processing
from tigramite.independence_tests import CMIknn
from tigramite.pcmci import PCMCI
import pandas as pd
import numpy as np

good_trials = ['DAP017(1)']
features = ["coughing", "pm2_5", "temperature", "humidity"]


for trial in good_trials:
    full_p = list()
    full_v = list()
    print("Trial: {}".format(trial))
    try:
        data = pd.read_csv(f"../../data/DAPHNE/PCMCI/{trial}.csv",
                                  infer_datetime_format=True, parse_dates=["timestamp"], index_col="timestamp")
        data = data[features]
        # Create a dataframe with the trial data
    except:
        print("ERROR")
        break
    # Create a dataframe with the trial data
    dataframe = data_processing.DataFrame(data.values, missing_flag=999.)
    # Initialise the non-linear CMIknn test
    cmi_knn = CMIknn(significance='shuffle_test', knn=0.1, shuffle_neighbors=5, transform='ranks', n_jobs=-1)
    # Initialise PCMCI
    pcmci = PCMCI(dataframe=dataframe, cond_ind_test=cmi_knn, verbosity=0)
    # For 7 lags in the first hour and every ten minutes afterwards up to the 8th hour
    #for lag in [1, 5, 10, 15, 30, 45, 60] + [i * 10 for i in range(7, 49)]:
    for lag in [i * 10 for i in range(34, 49)]:
        print("Time lag: {}".format(lag))
        # Use a RNG seed to reproduce results
        np.random.seed(0)
        # Run PCMCI
        results = pcmci.run_pcmciplus(tau_min=lag, tau_max=lag, pc_alpha=0.05)
        pvalues = results["p_matrix"][1][0][-1]
        stats = results["val_matrix"][1][0][-1]
        print("Lag {}".format(lag))
        print(pvalues)
        print(stats)
        full_p.append(pvalues)
        full_v.append(stats)

        df = pd.DataFrame(np.array(full_p))
        df.to_csv("results/p/non_linear_p_trial_{}_{}_8h.csv".format(trial, lag), index=False)

        df = pd.DataFrame(np.array(full_v))
        df.to_csv("results/v/non_linear_v_trial_{}_{}_8h.csv".format(trial, lag), index=False)

    try:
        # Save results
        df = pd.DataFrame(np.array(full_p).reshape(1, 49),
                          columns=[1, 5, 10, 15, 30, 45, 60] + [i * 10 for i in range(7, 49)])
        df.to_csv("results/p/non_linear_p_{}_8h.csv".format(trial), index=False)

        df = pd.DataFrame(np.array(full_v).reshape(1, 49),
                          columns=[1, 5, 10, 15, 30, 45, 60] + [i * 10 for i in range(7, 49)])
        df.to_csv("results/v/non_linear_v_{}_8h.csv".format(trial), index=False)

    except:
        continue

Trial: DAP017(1)
Time lag: 340
Lag 340
0.672
0.0038657494560814243
Time lag: 350
Lag 350
0.783
0.0031535076600128775
Time lag: 360
Lag 360
0.966
0.0004306718425777234
Time lag: 370
Lag 370
0.986
-0.0004379910678693477
Time lag: 380
Lag 380
0.911
0.0012733821402273193
Time lag: 390
Lag 390
0.972
0.00039672059178297303
Time lag: 400
Lag 400
0.68
0.00368457944759637
Time lag: 410
Lag 410
0.895
0.0016181357973472643
Time lag: 420
Lag 420
0.819
0.0030576184648083427
Time lag: 430
Lag 430
0.712
0.003534119645041578
Time lag: 440
Lag 440
0.693
0.00427660756286663
Time lag: 450
Lag 450
0.44
0.007148524748988194
Time lag: 460
Lag 460
0.969
0.000269291741903821
Time lag: 470
Lag 470
0.561
0.005934725708958588
Time lag: 480
Lag 480
0.764
0.003454409282969806
